In [1]:
#@title Mount the Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import sys
sys.path.insert(0, '/content/drive/MyDrive/seam1')
%cd /content/drive/MyDrive/seam1/


Mounted at /content/drive
/content/drive/MyDrive/seam1


In [2]:
#@title Load Image
import errno

import cv2
import logging
import os
import time

import numpy as np

from PIL import Image


def load_image(path):
    if not os.path.exists(path):
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)   # if the file is not found in the path, raise an exception

    if os.path.splitext(path)[1] == '.tif':                #special processing for tif files -- need to analyse
        img = np.asarray(Image.open(path), dtype=int)
        img[np.where(img == 0)] = 8
        img[np.where(img == 1)] = 0
        img = np.stack((img,)*3, axis=-1)
        img[:, :, 1] = 0
        img[:, :, 2] = 0
    else:
        img = cv2.imread(path)

    if img is None:
        raise Exception("Image is empty or corrupted", path)

    return img

def prepare_image(img, testing, cropping=True, vertical=False):
    # -------------------------------
    start = time.time()
    # -------------------------------

    if testing:                                    #if testing is true
        img[:, :, 0] = 0              #0,2 making as 0
        img[:, :, 2] = 0
        locations = np.where(img == 127)       # find the locations which is 127
        img[:, :, 1] = 0                       #make 1 also 0
        img[locations[0], locations[1]] = 255       # in the locations of where 127 is present do 255
        if cropping:                                 #if cropping required do other things --- not requries
            locs = np.array(np.where(img == 255))[0:2, ]
            img = img[np.min(locs[0, :]):np.max(locs[0, :]), np.min(locs[1, :]):np.max(locs[1, :])]

    else:
        ##################################################################
        # Protect against malformed inputs

        # Green channel should be empty
        assert len(np.unique(img[:, :, 1])) == 1
        assert np.unique(img[:, :, 1])[0] == 0

        # Red channel should have at most two values: 0 and 128 for boundaries
        assert len(np.unique(img[:, :, 2])) <= 2
        assert np.unique(img[:, :, 2])[0] == 0
        if len(np.unique(img[:, :, 2])) > 1:
            assert np.unique(img[:, :, 2])[1] == 128

        ##################################################################
        # Prepare the image

        # Find and remove boundaries: this is necessary as they are marked with 8 in the blue channel as well
        locations = np.where(img == 128)
        img[locations[0], locations[1]] = 0
        # Find regular text and text + decoration
        locations_text = np.where(img == 8)
        locations_text_decoration = np.where(img == 12)
        # Wipe the image
        locations_text = np.where(img == 8)
        locations_text_decoration = np.where(img == 12)
        # Set the text to be white
        img[locations_text[0], locations_text[1]] = 255
        img[locations_text_decoration[0], locations_text_decoration[1]] = 255

    # Rotate 90 degrees to the left the image (for vertical scripts such as Chinese)
    if vertical:
        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return img

In [3]:
#@title Preprocessing
import logging
import os
import sys
import time

import cv2
from skimage import measure

import matplotlib

#from src.line_segmentation.utils.util import save_img

matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np

def preprocess(image, small_component_ratio):
    # -------------------------------
    start = time.time()
    # -------------------------------

    # find the text area and wipe the rest
    image = wipe_outside_textarea(image)
    save_img(image, path=os.path.join('./output', 'after_wipe.png'), show=False)
    # Remove components which are too small in terms of area
    image = remove_small_components(image, small_component_ratio)
    save_img(image, path=os.path.join('./output', 'after_removesmall.png'), show=False)

    # Remove components which are too big in terms of area -> after removing the small ones!
    image = remove_big_components(image)
    save_img(image, path=os.path.join('./output', 'after_removebig.png'), show=False)

    image[image > 255] = 255

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------
    return image


def wipe_outside_textarea(image):

    # Save a copy of the original image
    ORIGINAL = image

    # Get only green channel
    image = image[:, :, 1]

    # SMOOTH IMAGE ######################################################################
    filter_size_H = 64
    filter_size_V = 192
    kernel = np.ones((filter_size_V, filter_size_H)) / filter_size_H
    # Apply averaging filter
    image = cv2.filter2D(image, -1, kernel)
    #SMOOTH_IMAGE = image
    # Draw a vertical line in the middle of the image to prevent 2 paragraphs to be split
    image[5:-5, int(image.shape[1] / 2) - 5:int(image.shape[1] / 2) + 5] = 255
    save_img(image, path=os.path.join('./output', 'smoothed_image_1.png'), show=False)
    # GET BIGGEST COMPONENT #############################################################
    # Get contour points of the binary polygon image
    tmp = np.ones((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    cc = measure.find_contours(image, 200, fully_connected='high')[0]
    # Swap the columns of cc as the coordinate are reversed
    cc[:, 0], cc[:, 1] = cc[:, 1], cc[:, 0].copy()
    # Cast to int to make, once again, cv2.fillPoly() happy
    cc = [cc.astype(np.int32, copy=False)]
    cv2.fillPoly(tmp, cc, (255, 255, 255))
    # DEBUG
    save_img(tmp, path=os.path.join('./output', 'smoothed_image.png'), show=False)

    # WIPE EVERYTHING OUTSIDE THIS AREA #################################################
    # Use 'tmp' as mask on the original image. Pixel with value '0' are text.
    tmp = tmp - ORIGINAL
    # Prepare image in RBG format s.t. we can use the coordinates systems of tmp
    image = np.stack((image,) * 3, axis=-1)
    # Wipe the pixels which are not selected by the mask
    image[np.where(tmp != 0)] = 0
    # DEBUG
    save_img(image, path=os.path.join('./output', 'filtered_image.png'), show=False)

    """
    # FILTER WITH VERTICAL PROJECTION PROFILE ###########################################
    # Compute projection profile
    ver = np.sum(SMOOTH_IMAGE, axis=0)
    # Get all values above average
    ver_indexes = np.where(ver > np.mean(ver))
    # Find the first and last of them
    left = np.min(ver_indexes)
    right = np.max(ver_indexes)

    # Wipe the image on left/right sides
    image[:, 0:left] = 0
    image[:, right:] = 0

    plt.figure()
    plt.plot(ver)
    plt.axhline(y=np.mean(ver), color='r', linestyle='-')
    plt.axvline(x=left, color='r', linestyle='-')
    plt.axvline(x=right, color='r', linestyle='-')
    plt.savefig('./output/ver.png')

    # FILTER WITH HORIZONTAL PROJECTION PROFILE ###########################################
    # Compute projection profile
    hor = np.sum(SMOOTH_IMAGE, axis=1)
    # Get all values above average
    hor_indexes = np.where(hor > np.mean(hor))
    # Find the first and last of them
    top = np.min(hor_indexes)
    bottom = np.max(hor_indexes)

    # Wipe the image on top/bottom sides
    image[0:top, :] = 0
    image[bottom:, :] = 0

    plt.figure()
    plt.plot(hor)
    plt.axhline(y=np.mean(hor), color='r', linestyle='-')
    plt.axvline(x=top, color='r', linestyle='-')
    plt.axvline(x=bottom, color='r', linestyle='-')
    plt.savefig('./output/hor.png')
    """
    return image


def remove_small_components(image, small_component_ratio):
    # Find CC
    cc_properties = measure.regionprops(measure.label(image[:, :, 1], background=0), cache=True)

    # Compute average metrics
    avg_area = np.mean([item.area for item in cc_properties])

    # Remove all small components
    for cc in cc_properties:
        if cc.area < small_component_ratio * avg_area:
            # Wipe the cc
            image[(cc.coords[:, 0], cc.coords[:, 1])] = 0
    return image


def remove_big_components(image):
    # Find CC
    cc_properties = measure.regionprops(measure.label(image[:, :, 1], background=0), cache=True)

    # Compute average metrics
    avg_area = np.mean([item.area for item in cc_properties])

    # Remove all small components
    for cc in cc_properties:
        if cc.area > 10 * avg_area:
            # Wipe the cc
            image[(cc.coords[:, 0], cc.coords[:, 1])] = 0
    return image



######################              BLUR IMAGE
import logging
import time

import cv2
import numpy as np


def blow_up_image(image, seams):
    # -------------------------------
    start = time.time()
    # -------------------------------

    # new image
    new_image = []

    # get the new height of the image and the original one
    ori_height, _, _ = image.shape
    height = ori_height + len(seams)

    seams = np.array(seams)

    for i in range(0, image.shape[1]):
        col = np.copy(image[:, i])
        y_cords_seams = seams[:, i, 1]

        seam_nb = 0
        for y_seam in y_cords_seams:
            col = np.insert(col, y_seam + seam_nb, [0, 0, 0], axis=0)
            seam_nb += 1

        new_image.append(col)

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return np.swapaxes(np.asarray(new_image), 0, 1), (((100 / ori_height) * height) - 100) / 100


def blur_image(img, save_name="blur_image.png", save=False, show=False, filter_size=1000, horizontal=True):
    # motion blur the image
    # generating the kernel
    kernel_motion_blur = np.zeros((filter_size, filter_size))
    if horizontal:
        kernel_motion_blur[int((filter_size - 1) / 2), :] = np.ones(filter_size)
    else:
        kernel_motion_blur[:, int((filter_size - 1) / 2)] = np.ones(filter_size)
    kernel_motion_blur = kernel_motion_blur / filter_size

    # applying the kernel to the input image
    output = cv2.filter2D(img, -1, kernel_motion_blur)

    if save:
        cv2.imwrite(save_name, output)

    if show:
        cv2.imshow('image', output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return output  # , np.sum(output, axis=2)



In [4]:
#@title Energy Map
import logging
import os
import sys
import time

import cv2
import numpy as np
from scipy.spatial import distance
from skimage import measure

#import src
#from src.line_segmentation.utils.unused_but_keep_them import blur_image
#from src.line_segmentation.utils.util import calculate_asymmetric_distance, save_img


def create_heat_map_visualization(ori_energy_map):
    # -------------------------------
    start = time.time()
    # -------------------------------

    heatmap = ((np.copy(ori_energy_map) / np.max(ori_energy_map)))
    heatmap = (np.stack((heatmap,) * 3, axis=-1)) * 255
    heatmap = np.array(heatmap, dtype=np.uint8)
    # show_img(cv2.applyColorMap(heatmap, cv2.COLORMAP_JET))
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    # result = cv2.add(cv2.applyColorMap(heatmap, cv2.COLORMAP_JET), img)

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return heatmap


def prepare_energy(ori_map, left_column, right_column, y):
    """
    Sets the left and right border of the matrix to int.MAX except at y.

    :param ori_map:
    :param y:
    :return:
    """

    y_value_left, y_value_right = left_column[y], right_column[y]
    ori_map[:, 0] = sys.maxsize / 2
    ori_map[:, -1] = sys.maxsize / 2

    ori_map[y][0], ori_map[y][-1] = y_value_left, y_value_right

    return ori_map

def create_distance_matrix(img_shape, centroids, asymmetric=False, side_length=1000):
    # -------------------------------
    start = time.time()
    # -------------------------------

    template = np.zeros((side_length, side_length))
    center_template = np.array([[np.ceil(side_length / 2), np.ceil(side_length / 2)]])
    pixel_coordinates = np.asarray([[x, y] for x in range(template.shape[0]) for y in range(template.shape[1])])

    # calculate distance template
    # TODO save template for speed up
    if asymmetric:
        template = np.array([calculate_asymmetric_distance(center_template, pxl, 1, 10) for pxl in pixel_coordinates]) \
            .flatten().reshape((side_length, side_length))
    else:
        template = distance.cdist(center_template, pixel_coordinates).flatten().reshape((side_length, side_length))

    # show_img(create_heat_map_visualization(template))

    distance_matrix = np.ones(img_shape) * np.max(template)
    # show_img(create_heat_map_visualization(template))
    # template[template > np.ceil(side_length / 2)] = np.max(template) * 2
    # round the centroid coordinates to ints to use them as array index
    centroids = np.rint(centroids).astype(int)

    # for each centroid
    for centroid in centroids:
        pos_v, pos_h = (centroid - np.ceil(side_length / 2)).astype(int)  # offset
        v_range1 = slice(max(0, pos_v), max(min(pos_v + template.shape[0], distance_matrix.shape[0]), 0))
        h_range1 = slice(max(0, pos_h), max(min(pos_h + template.shape[1], distance_matrix.shape[1]), 0))

        v_range2 = slice(max(0, -pos_v), min(-pos_v + distance_matrix.shape[0], template.shape[0]))
        h_range2 = slice(max(0, -pos_h), min(-pos_h + distance_matrix.shape[1], template.shape[1]))

        # need max
        distance_matrix[v_range1, h_range1] = np.minimum(template[v_range2, h_range2],
                                                         distance_matrix[v_range1, h_range1])

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    # distance_matrix = distance_matrix.reshape((centroids.shape[0], img_shape[0] * img_shape[1]))
    return distance_matrix.flatten()


def create_energy_map(img, blurring=True, projection=True, asymmetric=False):
    # -------------------------------
    start = time.time()
    # -------------------------------
    # get the cc, all the centroids and the areas of the cc
    cc, centroids, areas = find_cc_centroids_areas(img)

    # a list of all the pixels in the image as tuple
    centroids = np.asarray([[point[0], point[1]] for point in centroids])

    # normalise between 0 nd 1
    areas = (areas - np.min(areas)) / (np.max(areas) - np.min(areas))

    # bring it between -1 and 1
    areas = areas - np.mean(areas)

    # make it all negative
    areas = - np.abs(areas)

    # scale it with punishment
    areas *= 500

    # creating distance matrix
    # pixel_coordinates = np.asarray([[x, y] for x in range(img.shape[0]) for y in range(img.shape[1])])
    # distance_matrix = distance.cdist(pixel_coordinates, centroids[0:10])
    distance_matrix = create_distance_matrix(img.shape[0:2], centroids, asymmetric=asymmetric)

    # scale down the distance
    distance_matrix /= 30

    # make sure the distance is > 1
    distance_matrix += 1

    # We give all centroids the same energy (100)
    energy_background = ((np.ones(img.shape[0] * img.shape[1]) * 100) / distance_matrix).transpose()

    # Get the text location and assign it half the energy
    locs = np.array(np.where(img[:, :, 0].reshape(-1) == 0))[0:2, :]
    energy_text = energy_background
    energy_text[locs] = 0

    # sum up the received energy for each pixel
    energy_map = energy_background + energy_text
    #energy_map = energy_background / 100000
    energy_map = energy_map.reshape(img.shape[0:2])

    if blurring:
        # blur the map
        blurred_energy_map = blur_image(img=energy_map, filter_size=300)
        energy_map = blurred_energy_map

    if projection:
        projection_profile = create_projection_profile(energy_map)
        # scale it between 0 and max(energy_map) / 2
        projection_profile *= np.max(energy_map) / 2

        # blur projection profile
        projection_matrix = np.zeros(img.shape[0:2])
        projection_matrix = (projection_matrix.transpose() + projection_profile).transpose()
        projection_matrix = blur_image(projection_matrix, filter_size=1000)

        # overlap it with the normal energy map and add the text energy
        energy_map = energy_map + projection_matrix

    if True:
        # Cross-shaped kernel
        filter_size_H = img.shape[0]
        filter_size_V = img.shape[1]
        kernel = np.zeros((filter_size_V, filter_size_H))
        kernel[int(filter_size_V/2), :] = 1
        kernel[:, int(filter_size_H/2)] = 1

        # Apply cross filter
        smoothed = cv2.filter2D(energy_map, -1, kernel)

        # Smoothing kernel
        filter_size_H = 32
        filter_size_V = 32
        kernel = np.ones((filter_size_V, filter_size_H)) / (filter_size_V*filter_size_H)

        # Apply smoothing filter
        smoothed = cv2.filter2D(smoothed, -1, kernel)

        # Remove the mean and clip at 0
        smoothed -= np.mean(smoothed)
        smoothed[smoothed < 0] = 0

        # Normalize it between 0 and max(energy_map)
        smoothed = ((smoothed - np.min(smoothed)) * np.max(energy_map)) / (np.max(smoothed) - np.min(smoothed))

        # DEBUG
        #heatmap = src.line_segmentation.preprocessing.energy_map.create_heat_map_visualization(smoothed)
        #save_img(heatmap, path=os.path.join('./output/energy_map.png'))

        # Add it to the energy map
        energy_map = energy_map + smoothed

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return energy_map, cc


def create_projection_profile(energy_map):
    # creating the horizontal projection profile
    pp = np.sum(energy_map, axis=1)
    # smoothing it
    WINDOW_SIZE = 100
    pp = smooth(pp, WINDOW_SIZE)[int(WINDOW_SIZE/2):-int(WINDOW_SIZE/2-1)]
    # wipe everything below average
    pp -= np.mean(pp)
    pp[pp < 0] = 0
    # normalize it between 0-1
    pp = (pp - np.min(pp)) / (np.max(pp) - np.min(pp))
    return pp


def smooth(x, window_len=11, window='hanning'):
    """smooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also:

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s = np.r_[x[window_len - 1:0:-1], x, x[-2:-window_len - 1:-1]]
    # print(len(s))
    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = eval('np.' + window + '(window_len)')

    y = np.convolve(w / w.sum(), s, mode='valid')
    return y



def find_cc_centroids_areas(img):
    # -------------------------------
    start = time.time()
    # -------------------------------
    #############################################
    # Find CC
    cc_labels, cc_properties = get_connected_components(img)

    amount_of_properties = 0

    # Compute average metrics
    avg_area = np.mean([item.area for item in cc_properties])
    std_area = np.std([item.area for item in cc_properties])
    avg_height = np.mean([item.bbox[2] - item.bbox[0] for item in cc_properties])
    avg_width = np.mean([item.bbox[3] - item.bbox[1] for item in cc_properties])

    while amount_of_properties != len(cc_properties):
        # for _ in range(2):
        amount_of_properties = len(cc_properties)
        image = img[:, :, 1]
        #############################################
        # Cut all large components into smaller components
        coef = 1.5
        for item in cc_properties:
            if item.area > coef * avg_area \
                    or item.bbox[2] - item.bbox[0] > coef * avg_height \
                    or item.bbox[3] - item.bbox[1] > coef * avg_width:
                v_size = abs(item.bbox[0] - item.bbox[2])
                h_size = abs(item.bbox[1] - item.bbox[3])
                y1, x1, y2, x2 = item.bbox

                if float(h_size) / v_size > 1.5:
                    image[y1:y2, np.round((x1 + x2) / 2).astype(int)] = 0
                elif float(v_size) / h_size > 1.5:
                    image[np.round((y1 + y2) / 2).astype(int), x1:x2] = 0
                else:
                    # img[np.round((y1 + y2) / 2).astype(int), np.round((x1 + x2) / 2).astype(int)] = 0
                    image[y1:y2, np.round((x1 + x2) / 2).astype(int)] = 0
                    image[np.round((y1 + y2) / 2).astype(int), x1:x2] = 0

        img[:, :, 1] = image

        # Re-find CC
        cc_labels, cc_properties = get_connected_components(img)
        #############################################

    # Collect CC centroids
    all_centroids = np.asarray([cc.centroid[0:2] for cc in cc_properties])

    # Collect CC sizes
    all_areas = np.asarray([cc.area for cc in cc_properties])

    # Discard outliers & sort
    no_outliers = detect_outliers(all_areas, avg_area, std_area)
    centroids = all_centroids[no_outliers, :]
    filtered_area = all_areas[no_outliers]
    all_areas = filtered_area[np.argsort(centroids[:, 0])]
    all_centroids = centroids[np.argsort(centroids[:, 0]), :]

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return (cc_labels, cc_properties), all_centroids, all_areas


def get_connected_components(img):
    cc_labels = measure.label(img[:, :, 1], background=0)
    cc_properties = measure.regionprops(cc_labels, cache=True)
    return cc_labels, cc_properties


def detect_outliers(area, mean, std):
    # -------------------------------
    start = time.time()
    # -------------------------------
    if mean is not None:
        mean = np.mean(area)
    if std is not None:
        std = np.std(area)

    #no_outliers = abs(area - mean) < 3 * std
    no_outliers = area - 0.25*mean > 0

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return no_outliers

In [5]:
#@title Seam Carving
import itertools
import sys

import cv2
import numba
import numpy as np

#import src.line_segmentation

"""
    Code from: https://github.com/danasilver/seam-carving/blob/master/seamcarve.py
    homepage: http://www.faculty.idc.ac.il/arik/SCWeb/imret/
"""


@numba.jit()
def horizontal_seam(energies, penalty_reduction, bidirectional=False):
    """
    Spawns seams from the left to the right or from both directions. It returns the list of seams as point list.

    :param energies: the energy map
    :param penalty_reduction: if the penalty_reduction is smaller or equal to 0 we wont apply a penalty reduction
    :param bidirectional: if True there will be seams from left to right and right to left, else just from left to right
    :return: seams as point list
    """
    height, width = energies.shape[:2]
    # the y position we started (needed for the penalty)
    ori_y = 0
    # the last point we visit
    previous = 0
    # the points of the seam
    # from left to right
    seam_forward = []
    # from right to left
    seam_backward = []

    # spawns seams from left to right
    for i in range(0, width, 1):
        col = energies[:, i]
        if i == 0:
            ori_y = previous = np.argmin(col)
        else:
            top = col[previous - 1] if previous - 1 >= 0 else sys.maxsize
            middle = col[previous]
            bottom = col[previous + 1] if previous + 1 < height else sys.maxsize

            if penalty_reduction > 0:
                top += ((ori_y - (previous - 1)) ** 2) / penalty_reduction
                middle += ((ori_y - previous) ** 2) / penalty_reduction
                bottom += + ((ori_y - (previous + 1)) ** 2) / penalty_reduction

            previous = previous + np.argmin([top, middle, bottom]) - 1

        seam_forward.append([i, previous])

    # spawns seams from right to left
    if bidirectional:
        for i in range(width-1, -1, -1):
            col = energies[:, i]
            if i == width-1:
                ori_y = previous = np.argmin(col)
            else:
                top = col[previous - 1] if previous - 1 >= 0 else sys.maxsize
                middle = col[previous]
                bottom = col[previous + 1] if previous + 1 < height else sys.maxsize

                if penalty_reduction > 0:
                    top += ((ori_y - (previous - 1)) ** 2) / penalty_reduction
                    middle += ((ori_y - previous) ** 2) / penalty_reduction
                    bottom += + ((ori_y - (previous + 1)) ** 2) / penalty_reduction

                previous = previous + np.argmin([top, middle, bottom]) - 1

            seam_backward.append([i, previous])

    return [seam_forward, seam_backward[::-1]]


def draw_seams(img, seams, bidirectional=True):

    x_axis = np.expand_dims(np.array(range(0, len(seams[0]))), -1)
    seams = [np.concatenate((x, np.expand_dims(seam, -1)), axis=1) for seam, x in zip(seams, itertools.repeat(x_axis))]

    for i, seam in enumerate(seams):
        # Get the seam from the left [0] and the seam from the right[1]
        if bidirectional and i % 2 == 0:
            cv2.polylines(img, np.int32([seam]), False, (0, 0, 0), 3)  # Black
        else:
            cv2.polylines(img, np.int32([seam]), False, (255, 255, 255), 3)  # White


def draw_seams_red(img, seams, bidirectional=True):

    x_axis = np.expand_dims(np.array(range(0, len(seams[0]))), -1)
    seams = [np.concatenate((x, np.expand_dims(seam, -1)), axis=1) for seam, x in zip(seams, itertools.repeat(x_axis))]

    for i, seam in enumerate(seams):
        # Get the seam from the left [0] and the seam from the right[1]
            cv2.polylines(img, np.int32([seam]), False, (0, 0, 255), 3)  # Red


def get_seams(ori_energy_map, penalty_reduction, seam_every_x_pxl):
    # list with all seams
    seams = []
    # left most column of the energy map
    left_column_energy_map = np.copy(ori_energy_map[:, 0])
    # right most column of the energy map
    right_column_energy_map = np.copy(ori_energy_map[:, -1])
    # show_img(ori_enegery_map)
    for seam_at in range(0, ori_energy_map.shape[0], seam_every_x_pxl):
        energy_map = prepare_energy(ori_energy_map,left_column_energy_map, right_column_energy_map, seam_at)

        seams.extend(horizontal_seam(energy_map, penalty_reduction=penalty_reduction, bidirectional=True))

    # strip seams of x coordinate, which is totally useless as the x coordinate is basically the index in the array
    seams = np.array([np.array(s)[:, 1] for s in seams])

    return seams


def post_process_seams(energy_map, seams):
    # Check that the seams are as wide as the image
    assert energy_map.shape[1] == len(seams[0])

    # TODO implement a tabu-list to prevent two seams to repeatedly swap a third seam between them
    SAFETY_STOP = 100
    iteration = 0
    repeat = True
    while repeat:

        # Safety exit in case of endless loop meeting condition. See above.
        iteration += 1
        if iteration >= SAFETY_STOP:
            break

        repeat = False
        for index, seam_A in enumerate(seams):
            for seam_B in seams[index:]:
                # Compute seams overlap
                overlap = seam_A - seam_B

                # Smooth the overlap
                overlap[abs(overlap) < 10] = 0

                # Make the two seams really overlap
                seam_A[overlap == 0] = seam_B[overlap == 0]

                # Find non-zero sequences
                sequences = non_zero_runs(overlap)

                if len(sequences) > 0:
                    for i, sequence in enumerate(sequences):

                        target = sequence[1] - sequence[0]

                        left = sequence[0] - sequences[i - 1, 1] if i > 0 else sequence[0]
                        right = sequences[i + 1, 0] - sequence[1] if i < len(sequences)-1 else energy_map.shape[1] - sequence[1]

                        if target > left and target > right:
                            continue

                        repeat = True

                        # Expand the sequence into a range
                        sequence = range(*sequence)
                        # Compute the seam
                        energy_A = measure_energy(energy_map, seam_A, sequence)
                        energy_B = measure_energy(energy_map, seam_B, sequence)

                        # Remove the weaker seam sequence
                        if energy_A > energy_B:
                            seam_A[sequence] = seam_B[sequence]
                        else:
                            seam_B[sequence] = seam_A[sequence]

    return seams


def non_zero_runs(a):
    """
    Finding the consecutive non-zeros in a numpy array. Modified from:
    https://stackoverflow.com/questions/24885092/finding-the-consecutive-zeros-in-a-numpy-array
    """
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([1], np.equal(a, 0).view(np.int8), [1]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges


def measure_energy(energy_map, seam, sequence):
    """
    Compute the energy of that seams for the specified range
    """
    return energy_map[seam[sequence], sequence].sum()

In [6]:
#@title Binning
import itertools
import logging
import os
import time

import cv2
import numpy as np

#from src.line_segmentation.seamcarving_algorithm import draw_seams_red
#from src.line_segmentation.utils.util import calculate_asymmetric_distance, save_img


def majority_voting(connected_components, seams):
    """
    Splits the centroids into bins according to how many seams cross them
    """
    # -------------------------------
    start = time.time()
    # -------------------------------

    # Get the centroids and sort them
    centroids = np.asarray([cc.centroid[0:2] for cc in connected_components[1]])
    centroids = centroids[np.argsort(centroids[:, 0]), :]

    # for each centroid, compute how many seams are above it
    values = count_seams_below(centroids, seams)

    small_bins = [42]  # Just to enter the while loop once
    while len(small_bins) > 0:
        # split values into bins index
        bin_index, bin_size, unique_bins = split_into_bins_and_index(values)

        # look for outliers and merge them into bigger clusters
        if small_bins[0] == 42:
            avg = np.mean(bin_size[bin_size>1])*0.25

            # Compute average centroid horizontal distance:
            # distances = []
            # for bin in unique_bins:
            #     distances.extend(compute_avg_pairwise_distance(centroids[np.where(bin_index == bin)]))
            # threshold = 5 * np.mean(distances)
            #
            # # Scatter bins which have an anomaly in the avg distance
            # for bin in unique_bins:
            #     locs = np.where(bin_index == bin)
            #     if check_for_anomaly(centroids[locs], threshold):
            #         # Compute the offset for the scattered bin
            #         offset = np.array(range(0, len(locs[0])))
            #         # Assign them to the bin, thus scattering it into many single-centroid bins
            #         values[locs] -= offset
            #         # Get index of next bin
            #         nb = int(np.max(locs)) + 1
            #         # Adjust following bins accordingly
            #         values[nb:] -= np.max(offset)

        # Detect clusters which are too small
        small_bins = unique_bins[np.where(bin_size < avg)]

        # Merge small bins
        merge_small_bins(bin_index, centroids, small_bins, values)


    # Split the centroids into bins according to the clusters
    lines = []
    for bin in unique_bins:
        lines.append(list(centroids[np.where(bin_index == bin)]))

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return lines, centroids, values


def count_seams_below(centroids, seams):
    values = np.zeros([len(centroids)])
    for i, centroid in enumerate(centroids):
        cx = int(centroid[1])
        cy = int(centroid[0])
        for seam in seams:
            # if the seam is above the centroid at the centroid X position
            if seam[cx] > cy:
                values[i] = values[i] + 1
    return values


def merge_small_bins(bin_index, centroids, small_bins, values):
    for bin in small_bins:
        # find the indexes of the samples which are undernumbered
        for loc in np.where(bin_index == bin)[0]:
            # look for the next available bin below
            loc_p = loc + 1 if loc + 1 < len(values) else loc
            while bin_index[loc_p] == bin_index[loc]:
                if loc_p + 1 < len(values):
                    loc_p += 1
                else:
                    break

            # look for the next available bin above
            loc_m = loc - 1 if loc > 0 else loc
            while bin_index[loc_m] == bin_index[loc]:
                if loc_m > 0:
                    loc_m -= 1
                else:
                    break

            # compute distances to neighbors with the EUC distance
            XA = np.expand_dims(centroids[loc], axis=0)

            upper = np.array([calculate_asymmetric_distance(XA, c, 1, 5) for c in
                              centroids[np.where(bin_index == bin_index[loc_p])]]).min()
            lower = np.array([calculate_asymmetric_distance(XA, c, 1, 5) for c in
                              centroids[np.where(bin_index == bin_index[loc_m])]]).min()

            values[loc] = values[loc_m] if (upper == 0 or upper > lower) and lower != 0 else values[loc_p]


def split_into_bins_and_index(values):
    # Bin index is a an array with the bin number for each entry in values
    bin_index = np.digitize(values, np.unique(values))
    # Get the bins values and their size
    unique_bins, bin_size = np.unique(bin_index, return_counts=True)
    return bin_index, bin_size, unique_bins


def compute_avg_pairwise_distance(centroids):
    # Sort the centroids based on their x-coordinate
    centroids = centroids[np.argsort(centroids[:, 1]), :]
    dist = []
    for c1, c2 in pairwise(centroids):
        # Compute the distance in the horizontal axis
        dist.append(c2[1] - c1[1])
    return dist


def check_for_anomaly(centroids, threshold):
    # Sort the centroids based on their x-coordinate
    centroids = centroids[np.argsort(centroids[:, 1]), :]
    for c1, c2 in pairwise(centroids):
        # Compute the distance in the horizontal axis; If it is higher thant a threshold, trigger
        if c2[1] - c1[1] > threshold:
            return True
    return False


def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)


def draw_bins(img, centroids, root_output_path, seams, bins):
    # create white image
    binning_img = np.zeros(img.shape[0:2], dtype=np.uint8)
    binning_img.fill(255)
    # get text location
    locs = np.array(np.where(img[:, :, 0].reshape(-1) != 0))[0:2, :]
    binning_img = binning_img.flatten()
    binning_img[locs] = 211
    binning_img = binning_img.reshape(img.shape[0:2])
    binning_img = np.stack((binning_img,) * 3, axis=-1)
    # draw seams
    draw_seams_red(binning_img, seams)
    overlay_img = binning_img.copy()
    # draw the centroids on the seam energy map
    for centroid, value in zip(centroids, bins):
        cv2.circle(overlay_img, (int(centroid[1]), int(centroid[0])), 25, (0, 255, 0), -1)
        cv2.putText(binning_img, str(int(value)), (int(centroid[1]) - 16, int(centroid[0]) + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.addWeighted(overlay_img, 0.4, binning_img, 0.6, 0, binning_img)
    save_img(binning_img, path=os.path.join(root_output_path, 'energy_map', 'energy_map_bin_expl.png'))

In [7]:
#@title Polygon Manager
import logging
import time

import cv2
import networkx as nx1
import numpy as np
from skimage import measure

#from src.line_segmentation.utils.graph_util import createTINgraph, print_graph_on_img


def get_polygons_from_lines(img, lines, connected_components, vertical):




    # Extract the contour of each CC
    polygon_coords = []
    for i, line in enumerate(lines):
        cc_coords = []
        graph_nodes = []
        polygon_img = np.zeros(img.shape)
        for c in line:
            cc = find_cc_from_centroid(c, connected_components[1])
            points = cc.coords[::3, 0:2]
            points = np.asarray([[point[1], point[0]] for point in points])
            cc_coords.append(points)
            # add the first contour point to the list s.t. the line will be connected
            graph_nodes.append(find_graph_node(cc.coords, cc.centroid))

        # create graph
        overlay_graph = createTINgraph(np.array(list(set(tuple(p) for p in graph_nodes))))

        # create mst
        overlay_graph = nx.minimum_spanning_tree(overlay_graph)

        # overlay
        polygon_img = print_graph_on_img(polygon_img, [overlay_graph], color=(255, 255, 255), thickness=1)
        cv2.fillPoly(polygon_img, cc_coords, color=(255, 255, 255))

        # for cc_coord in cc_coords:
        #     # add cc areas to the image
        #     cv2.fillPoly(polygon_img, cc_coord, color=(255, 255, 255))

        if vertical:
            polygon_img = cv2.rotate(polygon_img, cv2.ROTATE_90_CLOCKWISE)

        # blurring the polygon image to close the gaps between the cut CCs
        filter_size_H = 5
        filter_size_V = 5
        kernel = np.ones((filter_size_V, filter_size_H)) / filter_size_H
        # Apply averaging filter
        polygon_img = cv2.filter2D(polygon_img, -1, kernel)

        # get contour points of the binary polygon image
        polygon_coords.append(measure.find_contours(polygon_img[:, :, 0], 5, fully_connected='high')[0])

    return polygon_coords


def find_graph_node(coords, centroid):
    # cast centroid coordinates into int
    centroid = np.asarray(centroid, dtype=int)

    # if centroid is in the coords we return the centroid
    if centroid in coords:
        return centroid

    # get the extreme points in coords on the same y as centroid
    # extrem_points = coords[np.where(coords[:, 1] == centroid[1])]

    return [coords[0][0], coords[0][1]]


def find_cc_from_centroid(c, cc_properties):
    #   c[0], c[1] = c[1], c[0]
    for cc in cc_properties:
        if cc.centroid[0] == c[0] and cc.centroid[1] == c[1]:
            return cc
    print("If this is printed, you might want to uncomment the line swapping the coordinates!")
    return None


def draw_polygons(image, polygons, vertical):
    if vertical:
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

    for polygon in polygons:
        cv2.polylines(image, np.array([[[np.int(p[1]), np.int(p[0])] for p in polygon]]), 1, color=(248, 24, 148), thickness=3)
    return image


def polygon_to_string(polygons):
    # -------------------------------
    start = time.time()
    # -------------------------------
    strings = []
    for polygon in polygons:
        line_string = []
        for i, point in enumerate(polygon):
            if i % 3 != 0:
                continue
            line_string.append("{},{}".format(int(point[1]), int(point[0])))
        strings.append(' '.join(line_string))

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------
    return strings

In [8]:
#@title Util - XML
import datetime
import re
import xml.dom.minidom as minidom
import xml.etree.cElementTree as ET

import numpy as np


def writePAGEfile(output_path, text_lines="", text_region_coords="not provided", baselines=None):
    # Create root element and add the attributes
    root = ET.Element("PcGts")
    root.set("xmls", "http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15")
    root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
    root.set("xsi:schemaLocation", "http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15 http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15/pagecontent.xsd")

    # Add metadata
    metadata = ET.SubElement(root, "Metadata")
    ET.SubElement(metadata, "Creator").text = "Michele Alberti, Vinaychandran Pondenkandath"
    ET.SubElement(metadata, "Created").text = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    ET.SubElement(metadata, "LastChange").text = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')

    # Add page
    page = ET.SubElement(root, "Page")

    # Add TextRegion
    textRegion = ET.SubElement(page, "TextRegion")
    textRegion.set("id", "region_textline")
    textRegion.set("custom", "0")

    # Add Coords
    ET.SubElement(textRegion, "Coords", points=text_region_coords)

    # Add TextLine
    for i, line in enumerate(text_lines):
        textLine = ET.SubElement(textRegion, "TextLine", id="textline_{}".format(i), custom="0")
        ET.SubElement(textLine, "Coords", points=line)
        if baselines:
            ET.SubElement(textLine, "Baseline", points=baselines[i])
        else:
            ET.SubElement(textLine, "Baseline", points="not provided")
        textEquiv = ET.SubElement(textLine, "TextEquiv")
        ET.SubElement(textEquiv, "Unicode")

    # Add TextEquiv to textRegion
    textEquiv = ET.SubElement(textRegion, "TextEquiv")
    ET.SubElement(textEquiv, "Unicode")

    #print(prettify(root))

    # Save on file
    file = open(output_path, "w")
    file.write(prettify(root))
    file.close()


def read_max_textline_from_file(pageFile):
    tree = ET.parse(pageFile)
    root = tree.getroot()
    NSMAP = {'pr': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}
    id = 0
    for textregion in root[1].findall('.//pr:TextRegion', namespaces=NSMAP):
        if 'textline' in textregion.attrib['id']:
            for textline in textregion.findall('.//pr:TextLine', namespaces=NSMAP):
                str = textline.attrib['id']
                id = np.max([id, int(re.findall(r'\d+', str)[0])])
    return id+1


def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [9]:
import shutil

import cv2
import os
import numpy as np


def create_folder_structure(input_file, output_path, params):
    """
    Creates the following folder structure:
    inputfilename_params
        - graph
        - histo

    :param input_file:
    :param output_path:
    :return:
    """
    fileName = os.path.basename(input_file).split('.')[0]

    # If the output_path does not exist (the output folder typically) then create it
    if not os.path.exists(output_path):
        os.mkdir(output_path)

    # basefolder
    basefolder_path = os.path.join(output_path, fileName + '_penalty_reduction_{}_seams_{}_component_ratio_{}'.format(*params))
    # create basefolder
    if not os.path.exists(basefolder_path):
        os.mkdir(basefolder_path)
        # create energy maps folder
        os.mkdir(os.path.join(basefolder_path, 'energy_map'))
        # create histo folder
        os.mkdir(os.path.join(basefolder_path, 'logs'))
        # create preprocess folder
        os.mkdir(os.path.join(basefolder_path, 'preprocess'))

    return basefolder_path


def save_img(img, path='experiment.png', show=False):
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    # Save the image at the given path
    cv2.imwrite(path, img)


def calculate_asymmetric_distance(x, y, h_weight=1, v_weight=5):
    return [np.sqrt((((y[0] - x[0][0]) ** 2) * v_weight + ((y[1] - x[0][1]) ** 2) * h_weight)/ (h_weight+v_weight))]


def dict_to_string(dictionay):
    string = []
    for entry in dictionay.items():
        string.append('_'.join(entry))
    return '_'.join(string)

In [10]:
#@title Util - Graph
import bisect
import logging
import os
import time

import networkx as nx
import numpy as np
import cv2

from scipy.spatial import Delaunay
from shapely.geometry import LineString

#from src.line_segmentation.utils.util import save_img


# Lars
# triangulate the CC; transform into a graph
# graph = createTINgraph(centroids)
#
# # use the seams to cut them into graphs
# graphs = cut_graph_with_seams(graph, last_seams, too_small_pc)
#
# GraphLogger.draw_graphs(img, graphs, name='cut_graph.png')
#


def createTINgraph(points):
    """
    http://ssrebelious.blogspot.com/2014/11/how-to-create-delauney-triangulation.html

    Creates a graph based on Delaney triangulation

    @param points: list of points
    @return - a graph made from a Delauney triangulation

    @Copyright notice: this code is an improved (by Yury V. Ryabov, 2014, riabovvv@gmail.com) version of
                      Tom's code taken from this discussion
                      https://groups.google.com/forum/#!topic/networkx-discuss/D7fMmuzVBAw
    """
    # -------------------------------
    start = time.time()
    # -------------------------------

    TIN = Delaunay(points)
    edges = set()
    # for each Delaunay triangle
    for n in range(TIN.nsimplex):
        # for each edge of the triangle
        # sort the vertices
        # (sorting avoids duplicated edges being added to the set)
        # and add to the edges set
        edge = sorted([TIN.vertices[n, 0], TIN.vertices[n, 1]])
        # TODO weighted eucleadean distance measure
        edges.add((edge[0], edge[1], asymetric_distance(edge, points)))
        edge = sorted([TIN.vertices[n, 0], TIN.vertices[n, 2]])
        edges.add((edge[0], edge[1], asymetric_distance(edge, points)))
        edge = sorted([TIN.vertices[n, 1], TIN.vertices[n, 2]])
        edges.add((edge[0], edge[1], asymetric_distance(edge, points)))

    # make a graph based on the Delaunay triangulation edges
    graph = nx.Graph()
    graph.add_weighted_edges_from(edges)

    original_nodes = points
    assert len(original_nodes) == graph.number_of_nodes()

    # create attribute dict
    attributes = {}
    for n in range(len(original_nodes)):
        XY = original_nodes[n]  # X and Y tuple - coordinates of the original points
        attributes[n] = [XY[1], XY[0]]

    # add attributes to graph
    nx.set_node_attributes(graph, attributes, 'XY')

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return graph


def asymetric_distance(edge, points):
    return np.linalg.norm(
        np.asarray(points[edge[0]]) * np.array([1, 3]) - np.asarray(points[edge[1]]) * np.array([1, 3]))


def print_graph_on_img(img, graphs, color=(0, 255, 0), thickness=3):
    img = img.copy()
    for graph in graphs:
        for edge in graph.edges:
            p1, p2 = get_edge_node_coordinates(edge, graph)
            cv2.line(img, p1, p2, color, thickness=thickness)

    return img


def get_edge_node_coordinates(edge, graph):
    # node attributes (in our case the XY attribute)
    node_attributes = nx.get_node_attributes(graph, 'XY')

    p1 = np.asarray(node_attributes[edge[0]], dtype=np.uint32)
    p1 = (p1[0], p1[1])
    p2 = np.asarray(node_attributes[edge[1]], dtype=np.uint32)
    p2 = (p2[0], p2[1])
    return p1, p2


def cut_graph_with_seams(graph, seams, too_small_pc):
    # -------------------------------
    start = time.time()
    # -------------------------------

    tic = time.time()
    unique_edges, weights, occurrences = find_intersected_edges(graph, seams)
    logging.info("find_intersected_edges: {}".format(time.time()-tic))

    # create histogram and save it
    # plt.hist(occurrences, bins='auto')
    # plt.savefig(os.path.join(output_path, 'histo/histogram_of_intersection_occurrences.png'))

    # remove the edges from the graph
    graph.remove_edges_from(unique_edges)

    if nx.is_connected(graph):
        return list([graph])

    # get the graphs
    graphs = np.asarray(list(nx.connected_component_subgraphs(graph)))
    # detect the small ones
    small_graphs = detect_small_graphs(graphs, too_small_pc).tolist()

    # check if there are small graphs
    while small_graphs:
        # merge the small graphs
        graph = merge_small_graphs(graph, list(small_graphs), unique_edges, weights)
        # get the graphs
        graphs = np.asarray(list(nx.connected_component_subgraphs(graph)))
        # detect the small ones
        small_graphs = detect_small_graphs(graphs, too_small_pc)

    # check again if the graph is still connected
    if nx.is_connected(graph):
        return list([graph])

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------

    return graphs


def get_neighbouring_seams_index(seams_max_y, seams_min_y, edge_max_y, edge_min_y):
    """
    Get the indexes of the seams which could possibly intersect with the point
    """
    # reads as "the highest (in terms of value, don't forget top left is 0,0!) value of
    # a seam has to be higher than the lowest value of the edge, and, the lowest point
    # of the seams has to be lower than the highest of the edge. In this way there CAN
    # be an intersection, but outside this range its mathematically impossible that there is.

    # IMPORTANT: note that seams_max_y and seams_min_y MUST be sorted. Here, we don't
    # sort them because of their sorted nature. As a matter of fact, it is impossible
    # for a seam to cross another one; if they meet the merge but one will never go on "the
    # other side" because of obvious reasons but most importantly the penalty. So the lists
    # are for us already sorted.
    return [bisect.bisect_left(seams_max_y, edge_min_y), bisect.bisect_left(seams_min_y, edge_max_y)]


def chunks(l, n):
    """Yield successive len(l)/n-sized chunks from l."""
    # ensure we dont ask for too many chunks
    n = np.min([n, len(l)])
    # get the size of each chunk
    size  = int(np.ceil(len(l)/n))
    for i in range(0, len(l), size):
        yield l[i:i + size]


def find_intersected_edges(graph, seams):
    # strip seams of x coordinate, which is totally useless as the x coordinate is basically the index in the array
    seams_y = [np.array(s)[:, 1] for s in seams]
    seams_max_y = np.max(seams_y, axis=1)
    seams_min_y = np.min(seams_y, axis=1)

    # node attributes (in our case the XY attribute)
    node_attributes = nx.get_node_attributes(graph, 'XY')

    # compute the lines out of seams
    seams = [LineString(seam) for seam in seams]

    # make the data iterable/splittable
    edges = [e for e in graph.edges]
    edges.sort(key=lambda tup: tup[0])

    edges_to_remove = []
    # split the data into chunks
    for chunk in chunks(edges, 250):
        # select the max/min of the edges in this chunk
        tmp = np.array(chunk)
        p1 = np.array([node_attributes[edge[0]] for edge in tmp])
        p2 = np.array([node_attributes[edge[1]] for edge in tmp])
        edge_max_y = np.max((p1[:, 1], p2[:, 1]))
        edge_min_y = np.min((p1[:, 1], p2[:, 1]))
        # get the possible seams to intersect
        index = get_neighbouring_seams_index(seams_max_y, seams_min_y, edge_max_y, edge_min_y)
        # for each edge in the chunk check if it actually does intersect
        for start, end, edge in zip(p1, p2, chunk):
            line_edge = LineString([start, end])
            for seam in seams[index[0]:index[1]]:
                if line_edge.intersects(seam):
                    edges_to_remove.append(edge)
                    break

    # # LEGACY CODE - not optimized but readable
    # edges_to_remove = []
    # for edge in graph.edges:
    #     p1 = np.asarray(node_attributes[edge[0]], dtype=np.uint32)
    #     p2 = np.asarray(node_attributes[edge[1]], dtype=np.uint32)
    #     line_edge =  LineString([p1, p2])
    #     for seam in seams:
    #         if line_edge.intersects(seam):
    #             edges_to_remove.append(edge)
    #             break

    # getting unique edges and counting them (how many times they where hit by a seam)
    unique_edges, occurrences = np.unique(np.array(edges_to_remove), return_counts=True, axis=0)
    weights = [graph.edges[edge]['weight'] for edge in unique_edges]

    return unique_edges, weights, occurrences


def merge_small_graphs(graph, small_graphs, unique_edges, weights):
    # list of edges to restore in the graph
    edges_to_add = []
    weights = np.asarray(weights)

    # TODO based on distance
    # merging small graph
    while small_graphs:
        # the indices of all edges which where cut from the given (small)graph
        small_graph = small_graphs.pop()
        edge_idxs = np.unique(np.hstack(np.asarray(
            [np.where(unique_edges == node)[0] for node in list(small_graph.nodes)])))
        # find the index in the in th edge index array
        min_edge_idx = edge_idxs[np.argmin(weights[edge_idxs])]
        # get edge to restore and add it to the list of edges to add
        edge = unique_edges[min_edge_idx]
        unique_edges = np.delete(unique_edges, min_edge_idx, axis=0)
        weights = np.delete(weights, min_edge_idx, axis=0)
        edges_to_add.append((edge[0], edge[1], weights[min_edge_idx]))
    # add again the edges
    graph.add_weighted_edges_from(edges_to_add)
    return graph


def graph_to_point_lists(graphs):
    return [list(nx.get_node_attributes(graph, 'XY').values()) for graph in graphs]


def detect_small_graphs(graphs, too_small_pc):
    # -------------------------------
    start = time.time()
    # -------------------------------

    graph_sizes = np.asarray([len(g.nodes) for g in graphs])
    # threshold which graphs are considered as small
    too_small = graph_sizes < too_small_pc * np.mean(graph_sizes)

    # -------------------------------
    stop = time.time()
    logging.info("finished after: {diff} s".format(diff=stop - start))
    # -------------------------------
    # return graphs[too_small]
    return graphs[too_small]


In [11]:
#@title Util - Graphlogger
import cv2
import os

import numpy as np

#from src.line_segmentation.utils.graph_util import get_edge_node_coordinates
#from src.line_segmentation.utils.util import save_img


class GraphLogger:
    IMG_SHAPE = ()
    ROOT_OUTPUT_PATH = ''

    @classmethod
    def draw_graphs(cls, img, graphs, color=(0, 255, 0), thickness=3, name='graph.png'):
        if not list(img):
            img = np.zeros(cls.IMG_SHAPE)
        else:
            img = img.copy()

        for graph in graphs:
            img = cls.draw_graph(img, graph, color, thickness, False)

        save_img(img, path=os.path.join(cls.ROOT_OUTPUT_PATH, 'graph', name), show=False)

        return img

    @classmethod
    def draw_graph(cls, img, graph, color=(0, 255, 0), thickness=3, save=False, name='graph.png'):
        if not list(img):
            img = np.zeros(cls.IMG_SHAPE)
        else:
            img = img.copy()

        cls.draw_edges(img, graph.edges, graph, color, thickness, save=False)

        if save:
            save_img(img, path=os.path.join(cls.ROOT_OUTPUT_PATH, 'graph', name), show=False)

        return img

    @classmethod
    def draw_edges(cls, img, edges, graph, color, thickness, save=False, name='graph.png'):
        for edge in edges:
            p1, p2 = get_edge_node_coordinates(edge, graph)
            cv2.line(img, p1, p2, color, thickness=thickness)

        if save:
            save_img(img, path=os.path.join(cls.ROOT_OUTPUT_PATH, 'graph', name), show=False)

In [12]:
#@title Upload Image
from google.colab import files
img = files.upload()

Saving test1.png to test1 (4).png


In [13]:
#@title Text Line Segmentation
# Utils
import logging
import os
import time
import argparse

#import src.line_segmentation.preprocessing.energy_map
#from src.line_segmentation.bin_algorithm import majority_voting, draw_bins
#from src.line_segmentation.polygon_manager import polygon_to_string, get_polygons_from_lines
#from src.line_segmentation.preprocessing.load_image import prepare_image, load_image
#from src.line_segmentation.preprocessing.preprocess import preprocess
#from src.line_segmentation.seamcarving_algorithm import draw_seams, get_seams, post_process_seams, draw_seams_red
#from src.line_segmentation.utils.XMLhandler import writePAGEfile
#from src.line_segmentation.utils.graph_logger import GraphLogger
#from src.line_segmentation.utils.util import create_folder_structure, save_img


#######################################################################################################################


def extract_textline(input_path: str, output_path: str, penalty_reduction: int, seam_every_x_pxl: int,
                     testing: bool, vertical: bool, console_log: bool, small_component_ratio: float):
    """
    Function to compute the text lines from a segmented image. This is the main routine where the magic happens
    """

    # -------------------------------
    start_whole = time.time()
    # -------------------------------

    ###############################################################################################
    # Load the image
    img = load_image(input_path)

    ###############################################################################################
    # Creating the folders and getting the new root folder
    root_output_path = create_folder_structure(input_path, output_path, (penalty_reduction, seam_every_x_pxl, small_component_ratio))

    # Init the logger with the logging path
    init_logger(root_output_path, console_log)

    # Init the graph logger
    GraphLogger.IMG_SHAPE = img.shape
    GraphLogger.ROOT_OUTPUT_PATH = root_output_path

    ###############################################################################################
    # Prepare image (filter only text, ...)
    img = prepare_image(img, testing=testing, cropping=False, vertical=vertical)
    # Pre-process the image
    save_img(img, path=os.path.join(root_output_path, 'preprocess', 'original.png'))
    img = preprocess(img, small_component_ratio)
    save_img(img, path=os.path.join(root_output_path, 'preprocess', 'after_preprocessing.png'))

  ###############################################################################################
    # Create the energy map
    energy_map, connected_components = create_energy_map(img,blurring=False,projection=False, asymmetric=True)
    # Visualize the energy map as heatmap
    heatmap =create_heat_map_visualization(energy_map)
    save_img(heatmap, path=os.path.join(root_output_path, 'energy_map', 'energy_map_without_seams.png'))

    ###############################################################################################
    # Get the seams
    seams = get_seams(energy_map, penalty_reduction, seam_every_x_pxl)
    # Draw the seams on the heatmap
    draw_seams(heatmap, seams)
    save_img(heatmap, path=os.path.join(root_output_path, 'energy_map', 'energy_map_with_seams.png'))
    # Post-process the seams
    seams = post_process_seams(energy_map, seams)
    # Draw the seams on the heatmap
    draw_seams_red(heatmap, seams)
    save_img(heatmap, path=os.path.join(root_output_path, 'energy_map', 'energy_map_postprocessed_seams.png'))

    ###############################################################################################
    # Extract the bins
    lines, centroids, values = majority_voting(connected_components, seams)

    # Draw the bins on a white gray image of the text with red seams
    draw_bins(img, centroids, root_output_path, seams, values)
    ###############################################################################################
    # Get polygons from lines
    polygons = get_polygons_from_lines(img, lines, connected_components, vertical)
    # Draw polygons overlay on original image
    save_img(draw_polygons(img.copy(), polygons, vertical), path=os.path.join(root_output_path, 'polygons_on_text.png'))

    ###############################################################################################
    # Write the results on the XML file
    writePAGEfile(os.path.join(root_output_path, 'polygons.xml'), polygon_to_string(polygons))

    ###############################################################################################
    # -------------------------------
    stop_whole = time.time()
    logging.info("finished after: {diff} s".format(diff=stop_whole - start_whole))
    # -------------------------------'''

    return


#######################################################################################################################
def init_logger(root_output_path, console_log):
    # create a logging format
    formatter = logging.Formatter(fmt='%(asctime)s %(filename)s:%(funcName)s %(levelname)-8s %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')
    # get the logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    # create and add file handler
    handler = logging.FileHandler(os.path.join(root_output_path, 'logs', 'extract_textline.log'))
    handler.setLevel(logging.INFO)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    if console_log:
        # create and add stderr handler
        stderr_handler = logging.StreamHandler()
        stderr_handler.formatter = formatter
        logger.addHandler(stderr_handler)

#######################################################################################################################




In [15]:
extract_textline(input_path='test1.png', output_path= './output', penalty_reduction = 6000, seam_every_x_pxl= 100,testing=True, vertical=False, console_log= False, small_component_ratio= 0.1)
logging.info('Terminated')









INFO:root:finished after: 0.07800126075744629 s
INFO:root:finished after: 1.5886080265045166 s
INFO:root:finished after: 0.00021719932556152344 s
INFO:root:finished after: 0.21899199485778809 s
INFO:root:finished after: 17.893927335739136 s
INFO:root:finished after: 18.9797146320343 s
INFO:root:finished after: 0.05893683433532715 s
<ipython-input-5-d01188e81256>:17: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "horizontal_seam" failed type inference due to: No implementation of function Function(<function argmin at 0x7dd047953010>) found for signature:
 
 >>> argmin(list(float64)<iv=None>)
 
There are 2 candidate implementations:
     - Of which 2 did not match due to:
     Overload in function 'array_argmin': File: numba/np/arraymath.py: Line 706.
       With argument(s): '(list(float64)<iv=None>)':
      Rejected as the implementation raised a specific error:
        TypingError: Failed in nopython mode pipeline (step: nopython f